In [1]:
#create blank Pandas dataframe to hold review data
import pandas as pd
pd_reviews = pd.DataFrame(columns = ['country', 'username', 'stars', 'date', 'title', 'version', 'text'])

In [3]:
from itunes_app_review_scraper import iTunesScraper
base_path = [path to the directory that will hold the CSV files]
reviews = iTunesScraper.get_reviews(974748812, base_path)      #(app id, country code (optional))

In [5]:
pd_reviews.head()

,country,username,stars,date,title,version,text
0,AR,Vvrage,5,2016-02-01,Great,2.1,I'm so in love with this app. Keep on the good...
1,AR,Milena mrllicovsky,5,2016-04-10,Muy buena apo,2.1,Lo único malo que tiene es que no podes guarda...
2,AR,dmantelli,5,2016-11-05,Excelente,2.3.4,Ahora se pueden compartir los GIF a Whatsapp.
3,AR,Xv3rs4,5,2017-02-08,Es GENIAL!,2.5.2,Los mensajes son más divertidos con estos Gifs
4,AR,dapezlo,5,2017-02-25,Muy buena app,2.5.3,Lo único debería reproducir solo los q selecci...


In [3]:
import glob, os
#used to set a temporary reference to the itunes_app_review_scraper module. This is needed to reference a dictionary
#of country codes and country ids in the config.py file
import sys
sys.path.insert(0, [path to the installed itunes_app_review_scraper module])
import config

files_processed = 0

os.chdir(basepath)
for file in glob.glob("*.csv"):
    for key, value in config.Config.COUNTRIES.items():                  #key == country code, value == countryid
        if value == file.split('_',1)[0]:                               #files are in countryid_pagenum format
            with open(basepath + file,'r', encoding='utf-8') as f:      #this just works better than using read_csv
                df = pd.read_csv(f,delimiter='|', quotechar='^')        #by itself, at least for unicode files
                df['country'] = key
                pd_reviews = pd_reviews.append(df, ignore_index=True)   #read csv files and append to dataframe
                files_processed +=1
                if files_processed % 100 == 0:
                    print ('Processed %d files so far...' % files_processed)
print ('Finished processing %d files!' % files_processed)

Processed 100 files so far...
Processed 200 files so far...
Finished processing 267 files!


In [4]:
#let's take a look at our results so far
pd_reviews.head()

,Unnamed: 0,country,date,stars,text,title,username,version
0,0.0,US,2015-03-26,5.0,Nice collection of gifs. Easy to search and send.,Awesome,Janakasoft,1
1,1.0,US,2015-03-26,5.0,I can't believe it. As if some stroke of luck ...,NO WAY!!,JonH2O,1
2,2.0,US,2015-03-26,5.0,Emojis are so dead to me. And this is the Jazz...,Tha bessss,mangoteeth,1
3,3.0,US,2015-03-28,5.0,Wene I got the app I dident let it access my p...,Help,WavyTuna991,1.1
4,4.0,US,2015-04-27,5.0,Insanely easy to use. Amazing selection of GIF...,Love this app,Nicsmith888,1.2


In [5]:
#drop the index column. we don't need it
pd_reviews.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
#the iTunes store creates duplicates of most US reviews and saves them to the Bangladesh, Ivory Coast, Maldives,
#Liechtenstein and Serbia country storefronts...
pd_reviews['is_dupe'] = pd_reviews.duplicated(subset=['username','stars','date','title','version','text'])

In [7]:
len(pd_reviews)

5395

In [8]:
#this counts the numbers of dupes in these country storefronts
pd_reviews['country'].loc[(pd_reviews['country']!='US') & (pd_reviews['is_dupe']==True)].value_counts()

MV    978
LI    978
BD    978
CI    375
RS    175
Name: country, dtype: int64

In [9]:
#drop the non-US dupes
pd_reviews = pd_reviews.query("is_dupe == False or country == 'US'")
#drop the is_dupe column. we don't need it any longer
pd_reviews.drop('is_dupe', axis=1, inplace=True)

In [10]:
pd_reviews.head()

,country,date,stars,text,title,username,version
0,US,2015-03-26,5.0,Nice collection of gifs. Easy to search and send.,Awesome,Janakasoft,1
1,US,2015-03-26,5.0,I can't believe it. As if some stroke of luck ...,NO WAY!!,JonH2O,1
2,US,2015-03-26,5.0,Emojis are so dead to me. And this is the Jazz...,Tha bessss,mangoteeth,1
3,US,2015-03-28,5.0,Wene I got the app I dident let it access my p...,Help,WavyTuna991,1.1
4,US,2015-04-27,5.0,Insanely easy to use. Amazing selection of GIF...,Love this app,Nicsmith888,1.2


In [11]:
#search/replace in pandas seems unnecessarily complicated. this is my best attempt (so far) to clean up the data
#1. remove | and ^ characters (which will be used as delimiters and quote marks)
#2. remove carriage returns (which PowerBI doesn't seem to handle gracefully)
for col in pd_reviews.columns.values:
    pd_reviews[col] = pd_reviews[col].replace('\|', ' ', regex=True)
    pd_reviews[col] = pd_reviews[col].replace('\^', ' ', regex=True)
    pd_reviews[col] = pd_reviews[col].replace('\r', ' ', regex=True)
    pd_reviews[col] = pd_reviews[col].replace('\n', ' ', regex=True)

In [12]:
#confirm that everything looks good before saving the combined results to CSV file for load to PowerBI
pd_reviews.head()

,country,date,stars,text,title,username,version
0,US,2015-03-26,5.0,Nice collection of gifs. Easy to search and send.,Awesome,Janakasoft,1
1,US,2015-03-26,5.0,I can't believe it. As if some stroke of luck ...,NO WAY!!,JonH2O,1
2,US,2015-03-26,5.0,Emojis are so dead to me. And this is the Jazz...,Tha bessss,mangoteeth,1
3,US,2015-03-28,5.0,Wene I got the app I dident let it access my p...,Help,WavyTuna991,1.1
4,US,2015-04-27,5.0,Insanely easy to use. Amazing selection of GIF...,Love this app,Nicsmith888,1.2


In [13]:
#save the results to disk
pd_reviews.to_csv(basepath + 'Strava_Final.csv', sep='|', quotechar='^', encoding='utf-8')